Install torch requires python 3.8.6 using below nightly build
pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118

Azure OpenAI limits the max input size 1.
embeddings = OpenAIEmbeddings(engine=<your deployment name>, chunk_size=1)

In [5]:

import os
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import UnstructuredMarkdownLoader

OPENAI_API_KEY= ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2022-12-01"
os.environ["OPENAI_API_BASE"] = "https://qianwenai.openai.azure.com"

persist_directory = "C:\\Users\\qianwens\\gpt\\db"
embeddings = OpenAIEmbeddings(chunk_size=1)
if(not os.path.exists(persist_directory)):
    loader = UnstructuredMarkdownLoader("C:\\Users\\qianwens\\tsg.md")
    data = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(data)
    db = Chroma.from_documents(docs, embeddings, persist_directory=persist_directory)   
    db.persist()
else:
    db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    



Using embedded DuckDB with persistence: data will be stored in: C:\Users\qianwens\gpt\db


In [10]:

llm = ChatOpenAI(
model_name="gpt-35-turbo",
temperature=0,
model_kwargs={"api_key": OPENAI_API_KEY, "api_base": "https://qianwenai.openai.azure.com", "api_type": "azure",
                                "api_version": "2023-03-15-preview", },
max_tokens=512,
verbose=True,
engine="gpt-35-turbo" 
)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

mysql_template = """As a service connector on azure tsg bot, your goal is to provide accurate
and helpful information about service connector on azure. You should answer user inquiries based on the
context provided and avoid making up answers. If you don't know the answer,
simply state that you don't know.

{context}

Question: {question}"""
prompt = PromptTemplate(
template=mysql_template, input_variables=["context", "question"]
)

bot = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=db.as_retriever(),
chain_type_kwargs={"prompt": prompt},
verbose=True,
)

while (True):
    query = input("Enter query: ")
    if query == "exit":
        break
    result = bot.run(query)
    print(result)

db = None



> Entering new RetrievalQA chain...

> Finished chain.
I'm sorry, I don't understand your question. Can you please provide more context or clarify your inquiry?


> Entering new RetrievalQA chain...

> Finished chain.
You can use the following command to reboot a node in AKS:

kubectl drain <node_name> --ignore-daemonsets && kubectl delete node <node_name>

This will gracefully evict all the pods running on the node and then delete the node. The node will then be recreated by AKS with a fresh OS image.
